In [ ]:
import numpy as np
import pandas as pd
from scripts.python.routines.manifest import get_manifest
from scripts.python.preprocessing.serialization.routines.pheno_betas_checking import get_pheno_betas_with_common_subjects
import pathlib
from tqdm.notebook import tqdm
from impyute.imputation.cs import fast_knn, mean, median, random, mice, mode, em


path = f"E:/YandexDisk/Work/pydnameth/datasets/meta/tasks/GPL13534_Blood/"
manifest = get_manifest('GPL13534')

disease = "Parkinson" # "Parkinson" "Schizophrenia"
data_type = "non_harmonized"

datasets = ["data_trn_val", "data_tst_GSE72774"]

path_load = f"{path}/{disease}/{data_type}"

df_trn_val = pd.read_pickle(f"{path_load}/data_trn_val.pkl")
df_tst = pd.read_pickle(f"{path_load}/data_tst_GSE72774.pkl")

In [ ]:
cpgs_trn_val_only = list(set(df_trn_val.columns.values) - set(df_tst.columns.values))
cpgs_tst_only = list(set(df_tst.columns.values) - set(df_trn_val.columns.values))

for f_id, f in enumerate(tqdm(cpgs_trn_val_only, desc=f"Imputation trn_val")):
    df_tst.loc[:, f] = df_trn_val[f].median()

for f_id, f in enumerate(tqdm(cpgs_tst_only, desc=f"Imputation tst")):
    df_trn_val.loc[:, f] = df_tst[f].median()


In [ ]:
df_all = pd.concat([df_trn_val, df_tst])

In [ ]:
cpgs_all = list(set(manifest.index.values).intersection(set(df_all.columns.values)))
is_nans = df_all.isnull().values.any()

In [ ]:
df_cpgs_selected = pd.read_excel(f"{path_load}/cpgs/43019.xlsx", index_col='features')
df_cpgs_selected = df_cpgs_selected.loc[df_cpgs_selected.index.isin(cpgs_all), :]
df_cpgs_selected.to_excel(f"{path_load}/cpgs/{df_cpgs_selected.shape[0]}_trn_val_tst_GSE72774.xlsx", index=True, index_label="features")